In [ ]:
#| default_exp simple_agent.network_architecture

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace

# Network Architecture

>  We define a custom Neural Network architecture to train our RL agent.

In [ ]:
#| export

from typing import Tuple

import torch
import torch.nn as nn
from d3rlpy.models.encoders import EncoderFactory

We define here 2 constants to represent the shape of the new numerical observations and the number of units in each hidden layer in our custom Deep Neural Network.

In [ ]:
#| export

HIDDEN_UNITS = 64

## CustomEncoder Class

We define a simple fully connected neural network to encode our environment observations.

In [ ]:
#| export


class SimpleEncoder(nn.Module):
    "Set up a custom Neural Network to train RL agent."

    def __init__(
        self,
        observation_shape: int,  # Environment observation shape, shape=(30,).
        feature_size: int,  # Number of network outputs (number of environment actions).
        hidden_units: int = HIDDEN_UNITS,  # Number of units in each hidden layer.
    ) -> None:
        "Init Neural Network architecture."
        # Initialize self._modules as OrderedDict.
        super(SimpleEncoder, self).__init__()

        self.feature_size = feature_size

        # First Layer.
        self.fc1 = nn.Linear(observation_shape, hidden_units)

        # 2nd Layer.
        self.fc2 = nn.Linear(hidden_units, feature_size)

    def forward(
        self,
        x: torch.Tensor,  # Numerical observation.
    ) -> torch.Tensor:
        "Process inputs"
        # Apply ReLU in each layer.
        h = torch.relu(self.fc1(x))
        h = torch.relu(self.fc2(h))
        return h

    # THIS IS IMPORTANT!(for EncoderFactory).
    def get_feature_size(self) -> int:
        "Returns the number of network outputs."
        return self.feature_size

In [ ]:
#| export


class SimpleEncoderFactory(EncoderFactory):
    "D3rlpy Custom Encoder."
    # This is necessary to override the EncoderFactory.
    TYPE = "custom"

    def __init__(
        self,
        feature_size: int,  # Number of network outputs (number of environment actions).
        observation_size: int,  # size of observation after transformation
        hidden_units: int = HIDDEN_UNITS,  # number of hidden
    ):
        "Init Encoder"
        self.feature_size, self.observation_size, self.hidden_units = (
            feature_size,
            observation_size,
            hidden_units,
        )

    def create(
        self,
        observation_shape: Tuple,  # Environment observation, shape
    ):
        # Return the custom DNN.
        return SimpleEncoder(self.observation_size, self.feature_size, self.hidden_units)

    def get_params(
        self,
        deep: bool = False,  # Flag to deeply copy objects.
    ):
        "Serialize neural network configuration."
        return {"feature_size": self.feature_size}

In [ ]:
#| hide
from nbdev import nbdev_export

nbdev_export()